In [1]:
import pandas as pd
import datetime, pytz, time
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt

import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_indicator import Indicators

C:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
def superreversal(High, Low, Close, fastema, slowema, stl1, stm1):
    ema1 = vbt.talib('EMA').run(Close, fastema).real.to_numpy()
    ema2 = vbt.talib('EMA').run(Close, slowema).real.to_numpy()
    st1 = vbt.pandas_ta('supertrend').run(High, Low, Close, stl1, stm1).supertd.to_numpy()

    entries = (ema1 > ema2) & (st1 > 0) & (Low < ema1)
    exits = (st1 < 0) & (High > ema1)

    return entries,  exits

SuperReversal = vbt.IndicatorFactory(
    class_name= 'SuperRevesal',
    short_name = 'SR',
    input_names = ["High", "Low", "Close"] ,
    param_names = ["fastema", "slowema", "stl1", "stm1"],
    output_names= ["entries", "exits"]
    ).from_apply_func(superreversal)




In [3]:
params = {
    'backtest': {
        'start': '2017-10-1',
        'end': '2018-1-1',
        'length': {
            'weeks': 4  #durée pendant laquelle on optimise les paramètres
        },
        'forward': {
            'weeks': 2  #durée pendant laquelle on joue les paramètres optimisés
        },
        
    },
    'strat':{
        'download_params': {
            'symbols': 'BTCUSDT',
            'interval': '1h',            
        },
        'get_params': {
            'column': ['High', 'Low', 'Close']
        },
        'run_params': { 
            'fastema': {'start': 10, 'stop': 30, 'step': 1},
            'slowema': {'start': 100, 'stop': 400, 'step': 20},
            'stl1': {'start': 15, 'stop': 17, 'step': 1},
            'stm1': {'start': 2, 'stop': 4, 'step': 1},
            'param_product': 'True'
        }, 
    },
    'portfolio': {'freq': 'H' ,
                  'fees' : 0.1/100,
                  'size_type': 'Percent',
                  'size': .1} #proportion du portefeuille à investir dans chaque trade
}



In [4]:
indics = Indicators
test = ForwardTest(indics.SuperReversal, params)
test.backtest(init_cash=1000)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-10-29 00:00:00+00:00 and 2017-11-12 00:00:00+00:00 finished at 2023-07-19 17:53:47.417973
Cash: 1000.5504298805788


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-12-10 00:00:00+00:00 and 2017-12-24 00:00:00+00:00 finished at 2023-07-19 17:58:04.922409
Cash: 1045.1606115817203


0it [00:00, ?it/s]

1045.1606115817203

In [5]:
pf = test.aggregate_results ()
pf.stats()

NameError: name 'stack' is not defined